# Extrai plano de aula da PGLS

Esse arquivo tem como função fazer um web scraping (pegar informações de uma página web) nos planos de aula da Pós Graduação Lato Sensu. Para isso utiliza-se o módulo requests do python para pedir os arquivos das páginas ao servidor (um por vez, começando no id mais alto e decrescendo até um número obtido empiricamente) e utiliza o beautiful soup para extrair as informações dessa página. Em um primeiro momento, os dados extraídos são: id, ano, período (31, 32, 33, 34), curso, disciplina, código da turma e link do plano. Esses dados são escritos em um arquivo .csv com a nomenclatura **planos_pgls_{start}_{end}.csv**.

## Como utilizar

Para rodar o script desse arquivo, garanta que criou um ambiente virtual com as dependências do requirements.txt e execute o arquivo todo.

### Importa os módulos necessários

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

### Função que faz as requisições, o web scraping da página e salva as informações em um csv 

In [1]:
def search_plans():
    classes_registered = [] # Cria uma lista que armazena as turmas já registradas no csv para evitar as repetições
    start = 61766 # Id mais alto dos planos de aula
    end = 0 # Id do ultimo plano de aula do ano de 2024 (obtido empiricamente)
        
    with open(f'planos_pgls_{start}_{end}.csv', 'w') as f:
        f.write(f'id,ano,periodo,curso,disciplina,codigo_turma,professores,email_professores,link\n') # Escreve o cabeçalho no csv
    
    # O TQDM é uma biblioteca opcional que mostra o progresso de um for loop
    for i in tqdm(range(start, end, -1)):
        # Faz uma requisição para obter a página HTML
        page = requests.get(f'https://cgi.insper.edu.br/DiarioDeClasse/PlanoDeAula/PlanoDeAulaPDF.aspx?t=Plano%20de%20Aula&id={i},&d=false')
        # Faz um parse do conteúdo da página
        soup = BeautifulSoup(page.content, "html.parser", from_encoding='utf-8')
        # Procura por todos os elementos da página que são uma div que possuem a classe textInline
        elements = soup.find_all('div', 'textInline')

        # Se a página possui um elemento (evita erro em páginas vazias)...
        if len(elements) > 0:
            course = elements[0].text.strip() # Extrai o curso referente ao plano
            subject = elements[1].text.strip() # Extrai a disciplina referente ao plano
            class_code = elements[2].text.strip() # Extrai o código da turma referente ao plano
            year, period = elements[4].text.strip().split(' / ') # Extrai o ano e o período referente ao plano
            teachers_names = elements[5].text.strip().split('/') # Extrai os nomes dos professores e os coloca em uma lista
            teachers_names = [' '.join([name.capitalize() for name in teacher.strip().split()]) for teacher in teachers_names] # Capitaliza os nomes
            teachers_names = ', '.join(teachers_names) # Junta os nomes em uma string
            teachers_emails = elements[6].text.strip().split('/') # Extrai os emails dos professores e os coloca em uma lista
            teachers_emails = [teacher.strip().lower() for teacher in teachers_emails] # Coloca os emails em caixa baixa
            teachers_emails = ', '.join(teachers_emails) # Junta os nomes em uma string
            
            # Se o período estiver no padrão da pós-graduação (exclui os planos da graduação), 
            # o código da turma não está na lista de códigos registrados e o ano é de 2024...
            if period in ['31', '32', '33', '34'] and class_code not in classes_registered and year == '2024':
                classes_registered.append(class_code) # Adiciona o código da turma na lista
                # Escreve no csv criado as informações do plano de aula
                with open(f'planos_pgls_{start}_{end}.csv', 'a', encoding='utf-8') as f: 
                    f.write(f'{i},{year},{period},"{course}","{subject}","{class_code}","{teachers_names}","{teachers_emails}","https://cgi.insper.edu.br/DiarioDeClasse/PlanoDeAula/PlanoDeAulaPDF.aspx?t=Plano%20de%20Aula%20Atualizado&id={i},&d=false"\n')
        

### Executa a função

In [35]:
search_plans()

100%|██████████| 21766/21766 [1:52:57<00:00,  3.21it/s]   
